# These are not LaTeX slides

In [1]:
import Pkg
Pkg.activate("..")

using SCPToolbox
using ECOS

  Activating project at `~/GitHub/SCPToolbox_tutorial`


## Objective function

 - This section explains how the __objective function__ is defined. 

 - Regardless of whether optimization problems have constraints or not, they seek to optimize a function with respect to  a chosen _objective_.

 - i.e., they seek to either minimize a certain _cost_ or maximize a certain _reward_. 

 - For example, propellant consumption is an important metric that is ideally optimized for (minimized) in a rocket-powered planetary landing mission. We may also want to minimize the total energy to go from one point to another.

__Note__: Although we take cost-minimization to be the default objective here, a problem can be easily cast as a reward-maximization problem by minimizing the negative of the objective function.

Let us define a new optimization problem, but this time, with a user-defined constant parameter, `a`.

In [2]:
my_pars = Dict("a" => 5)
opts = Dict("verbose" => 0)
pbm = ConicProgram(my_pars; solver = ECOS.Optimizer, solver_options = opts)

Conic linear program

  Feasibility problem
  0 variables (0 blocks)
  0 parameters (0 blocks)
  0 constraints

  Variable argument
    0 elements
    0 blocks


  Parameter argument
    0 elements
    0 blocks


Next, we define a scalar variable, `x`.

In [3]:
x = @new_variable(pbm, "x")

Vector variable
  1 elements
  (1,) shape
  Name: x
  Block index in stack: 1
  Indices in stack: 1
  Type: Vector{JuMP.AffExpr}
  No scaling (x=xh)
  Any perturbation allowed
  Value =
     x

Now we are ready to define our objective function, which we will henceforth refer to as simply "cost".

By default, the cost is initialized to zero when a new problem is generated. We use the `@add_cost` function to add terms to the cost `obj`.

In [4]:
obj = @add_cost(
                pbm, (x,), 
                begin
                    x, = arg
                    pars["a"]*x
                end
                )

Cost function composed of 1 terms

Term 1:
  Coefficient: 1.00e+00
  Affine function
  Arguments:
    x (block 1) : 1
  Current value =
    5 x

 - The first argument of the function is to indicate that we are adding the cost term to the problem we have defined, `pbm`. 

 - The third argument unpacks the tuple into variables that are used to define the cost. Here, our cost is simply the constant parameter that we defined earlier, `a`, and the cost itself is `a*x`.

 - The second argument is a tuple including the variables associated with the cost, which in this case is just `x`.

The cost function we have included is as follows:

In [5]:
cost(pbm)

Cost function composed of 1 terms

Term 1:
  Coefficient: 1.00e+00
  Affine function
  Arguments:
    x (block 1) : 1
  Current value =
    5 x

# Solve problem

With everything we have learned so far, we are now ready to solve a convex optimization problem!

We will start with a new problem for this example.

As before, we begin by defining constant parameters to be passed to the solver.

In [6]:
problem_pars = Dict("a" => 5, "x_ref" => [2; 2; 5; 10; -1])

Dict{String, Any} with 2 entries:
  "x_ref" => [2, 2, 5, 10, -1]
  "a"     => 5

Here, we have defined one scalar constant, `a`, and one 5-dimensional constant vector, `x_ref`.

Now, we create a dictionary of options to be passed to the low-level optimizer. In this case, we set the `verbose` flag to `1` so that we can see the full output of the low-level optimizer.

In [7]:
opts = Dict("verbose" => 1)

Dict{String, Int64} with 1 entry:
  "verbose" => 1

We now define our optimization problem. We shall make use of the open-source low-level convex optimization solver, ECOS.

In [8]:
pbm = ConicProgram(problem_pars;
    solver = ECOS.Optimizer,
    solver_options = opts)

Conic linear program

  Feasibility problem
  0 variables (0 blocks)
  0 parameters (0 blocks)
  0 constraints

  Variable argument
    0 elements
    0 blocks


  Parameter argument
    0 elements
    0 blocks


We define two variables, `t` of dimension 1, and `x` of dimension 5.

In [9]:
x = @new_variable(pbm, length(problem_pars["x_ref"]), "x")
t = @new_variable(pbm, "t")

Vector variable
  1 elements
  (1,) shape
  Name: t
  Block index in stack: 2
  Indices in stack: 6
  Type: Vector{JuMP.AffExpr}
  No scaling (x=xh)
  Any perturbation allowed
  Value =
     t

Now we add a second-order cone (SOC) constraint, $\|x-x_{ref}\|_2 \le t$, to the problem.

In [10]:
@add_constraint(
    pbm, SOC, (x, t),
    begin
        x, t = arg
        vcat(t, x-pars["x_ref"])
    end)

Name: f1
Cone f(x,p)∈K, where:
K is a second-order cone, {(t, x)∈ℝ×ℝⁿ : ‖x‖₂≤t}
f(x,p) = 
  t
  x[1] - 2
  x[2] - 2
  x[3] - 5
  x[4] - 10
  x[5] + 1
Affine function
Arguments:
  x (block 1) : 1:5
  t (block 2) : 6


We choose the cost function to be $t^{2}$.

In [11]:
@add_cost(
    pbm, (t,), 
    begin
        t, = arg
        t.^2
    end)

Cost function composed of 1 terms

Term 1:
  Coefficient: 1.00e+00
  Quadratic function
  Arguments:
    t (block 2) : 6
  Current value =
    t²

We are finally ready to solve the problem we just defined!

In [12]:
exit_status = solve!(pbm)


ECOS 2.0.8 - (C) embotech GmbH, Zurich Switzerland, 2012-15. Web: www.embotech.com/ECOS

It     pcost       dcost      gap   pres   dres    k/t    mu     step   sigma     IR    |   BT
 0  +0.000e+00  -2.449e-01  +4e+00  2e-01  1e-01  1e+00  2e+00    ---    ---    1  1  - |  -  - 
 1  -6.147e-02  -8.099e-02  +2e-01  7e-03  4e-03  3e-02  9e-02  0.9530  5e-03   1  1  1 |  0  0
 2  -5.548e-04  -2.077e-03  +1e-02  3e-04  2e-04  9e-04  5e-03  0.9475  8e-04   1  1  1 |  0  0
 3  -4.848e-04  -6.068e-04  +2e-03  6e-05  4e-05  4e-04  1e-03  0.9090  1e-01   1  2  2 |  0  0
 4  -4.348e-05  -7.855e-05  +8e-04  2e-05  1e-05  1e-04  4e-04  0.7244  1e-01   1  2  2 |  0  0
 5  -1.174e-04  -1.268e-04  +3e-04  9e-06  5e-06  7e-05  2e-04  0.8239  3e-01   1  1  1 |  0  0
 6  -1.708e-06  -3.017e-06  +5e-05  1e-06  7e-07  9e-06  2e-05  0.9890  1e-01   2  2  2 |  0  0
 7  -6.230e-08  -1.239e-07  +2e-06  6e-08  4e-08  4e-07  1e-06  0.9600  8e-03   1  1  1 |  0  0
 8  -5.661e-08  -6.196e-08  +2e-07  5e-09  3e-

OPTIMAL::TerminationStatusCode = 1

The optimal cost value is `J_opt`.

In [13]:
J_opt = objective_value(pbm)

-8.29959449100464e-10

The optimal values of the decision variables are `x_opt` and `t_opt`.

In [14]:
x_opt = value(x)

5-element Vector{Float64}:
  2.0000000000095266
  2.0000000000095266
  5.0000000000238165
 10.000000000047633
 -1.000000000004763

In [15]:
t_opt = value(t)

1-element Vector{Float64}:
 1.1524313115706931e-5

The optimal values of the decision variables can also be accessed as mentioned before.

In [16]:
value(variables(pbm, "^x\$"))

5-element Vector{Float64}:
  2.0000000000095266
  2.0000000000095266
  5.0000000000238165
 10.000000000047633
 -1.000000000004763

In [17]:
value(variables(pbm, "^t\$"))

1-element Vector{Float64}:
 1.1524313115706931e-5

Congratulations! 

We now know how to solve convex optimization problems, which form the basis for Sequential Convex Programming (SCP) problems, which we shall now get into with some exciting examples!